In [1]:
pip install pandas scikit-learn tensorflow fastapi uvicorn[standard]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.8 MB/s eta 0:00:00


In [6]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from fastapi import FastAPI
from pydantic import BaseModel

# Load the dataset (assuming the dataset is in JSON format)
# Replace 'dataset.json' with the actual file path
data = pd.read_json('dataset.json')

# Print column names
print(data.columns)

# Data Preprocessing
# Assuming the dataset has 'external_status' and 'internal_status' columns
# Cleaning and formatting the external status descriptions
data['externalStatus'] = data['externalStatus'].apply(lambda x: x.lower())  # Convert to lowercase
data['externalStatus'] = data['externalStatus'].str.replace('[^a-zA-Z\s]', '')  # Remove non-alphabetic characters

# Encode internal status labels
label_encoder = LabelEncoder()
data['internal_status_encoded'] = label_encoder.fit_transform(data['internalStatus'])

# Split data into features and target
X = data['externalStatus']
y = data['internal_status_encoded']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Convert external status descriptions to numerical data using Tokenization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Tokenize the external status descriptions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure uniform length
max_length = max([len(seq) for seq in X_train_sequences])
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post')

# Define the TensorFlow model
model = Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train, epochs=10, batch_size=32, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print("Accuracy:", accuracy)
# Define the TensorFlow model
model = Sequential([
    Dense(64, activation='relu', input_shape=(1,)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


# API Development
# Define FastAPI app
app = FastAPI()

# Define request body model
class Item(BaseModel):
    external_status: str

# Define API endpoint
@app.post("/predict/")
async def predict_internal_status(item: Item):
    external_status = item.external_status.lower().replace('[^a-zA-Z\s]', '')
    encoded_status = label_encoder.transform([external_status])
    prediction = model.predict(encoded_status)
    predicted_label = label_encoder.inverse_transform([np.round(prediction)])
    return {"predicted_internal_status": predicted_label[0]}

# Testing and Validation:
# Test the API thoroughly to ensure functionality and accuracy.

# Documentation:
# Comments within the code explain each step and provide context for future reference.


Index(['externalStatus', 'internalStatus'], dtype='object')
Epoch 1/10
31/31 [==============================] - 1s 1ms/step - loss: -1.3490 - accuracy: 0.2323
Epoch 2/10
31/31 [==============================] - 0s 1ms/step - loss: -9.3675 - accuracy: 0.2323
Epoch 3/10
31/31 [==============================] - 0s 1ms/step - loss: -31.4550 - accuracy: 0.2323
Epoch 4/10
31/31 [==============================] - 0s 1ms/step - loss: -79.6308 - accuracy: 0.2323
Epoch 5/10
31/31 [==============================] - 0s 1ms/step - loss: -168.9202 - accuracy: 0.2323
Epoch 6/10
31/31 [==============================] - 0s 1ms/step - loss: -312.5288 - accuracy: 0.2323
Epoch 7/10
31/31 [==============================] - 0s 1ms/step - loss: -525.8356 - accuracy: 0.2323
Epoch 8/10
31/31 [==============================] - 0s 1ms/step - loss: -826.3068 - accuracy: 0.2323
Epoch 9/10
31/31 [==============================] - 0s 1ms/step - loss: -1222.4733 - accuracy: 0.2323
Epoch 10/10
8/8 [===================

In [8]:
pip install uvicorn
